In [9]:
# SI - en intialize_problem_instances_nNLP: Comprobar si en la inicialización del entorno, para la capa de optimización se está muestreando a cada hora, debería ser así y luego remuestrear interpolando a la que haga falta
# TODO: Para la simulación, hay que usar el entorno (i.e. variables climáticas) del dataset con su frecuencia original resampleada a la del modelo
# TODO: Para la capa de optimización de la operación, se deben remuestrear las variables climáticas de la próxima hora a un tiempo de 10 min y baja incertidumbre, a partir de ahí se remuestrean cada hora y la incertidumbre normal 


In [13]:
from pathlib import Path
import datetime

debug_mode = False

%load_ext autoreload
%autoreload 2

base_path = Path("/workspaces/SolarMED")
date_span=["20180921", "20180922"]
env_date_span = ["20180921", "20180928"]
evaluation_id="dev"

file_id = f"results_nNLP_op_plan_eval_at_{datetime.datetime.now():%Y%m%dT%H%M}_{evaluation_id}"
output_path = Path(base_path) / f"optimization/results/{date_span[0]}_{date_span[1]}/{file_id}.h5"
output_path.parent.mkdir(parents=True, exist_ok=True)

# main(
# date_span=date_span,       
data_path= base_path / "optimization/data"
# env_date_span=env_date_span,
# output_path=output_path,
uncertainty_factor=0
operation_optimization_layer=False
# )



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Evaluate operation plan - startup / shutdown
from typing import Literal, get_args, Optional
import copy
from dataclasses import asdict, fields
from pathlib import Path
import time
import numpy as np
import pandas as pd
from loguru import logger
import pygmo as pg
import threading
import datetime
from tqdm.auto import tqdm  # notebook compatible

from solarmed_optimization.utils import times_to_samples
from solarmed_optimization.utils.serialization import get_fitness_history
from solarmed_optimization.problems import BaseNlpProblem
from solarmed_optimization.utils.progress import update_bar_every
from solarmed_optimization import (
    EnvironmentVariables,
    ProblemParameters,
    RealDecisionVariablesUpdatePeriod,
    InitialDecVarsValues,
    ProblemData,
    IrradianceThresholds,
    OptimToFsmsVarIdsMapping
)
from solarmed_optimization.problems.nlp import (
    AlgoParams, 
    ProblemsEvaluationParameters,
    OperationPlanResults,
    OpPlanActionType,
    batch_export
)
from solarmed_optimization.utils.initialization import (
    problem_initialization,
    InitialStates,
    initialize_problem_instances_nNLP
)
from solarmed_optimization.utils.evaluation import (evaluate_idle_thermal_storage,
                                                    evaluate_model)
from solarmed_optimization.utils.operation_plan import generate_update_datetimes


In [16]:
def get_initial_states(sim_df: Optional[pd.DataFrame] = None, Tts_h: Optional[list[float]] = None, Tts_c: Optional[list[float]] = None,) -> InitialStates:
    
    if sim_df is not None:
        Tts_h = [sim_df.iloc[-1][f"Tts_h_{key}"] for key in ["t", "m", "b"]]
        Tts_c = [sim_df.iloc[-1][f"Tts_c_{key}"] for key in ["t", "m", "b"]]
    elif Tts_h is None or Tts_c is None:
        Tts_h=[90, 80, 70]
        Tts_c=[70, 60, 50]
        
    return InitialStates(Tts_h=Tts_h, Tts_c=Tts_c)

def select_best_alternative(df: pd.DataFrame, std_penalty_weight: float =1.0, worst_case_penalty_weight: float =1.0) -> tuple[int, pd.DataFrame]:
    """
    Select the best alternative based on average performance, consistency (std dev), and worst-case scenario.
    
    Parameters:
    - df: pd.DataFrame
        Rows = alternatives, Columns = scenarios
    - std_penalty_weight: float
        How much to penalize alternatives with high standard deviation
    - worst_case_penalty_weight: float
        How much to penalize based on the worst-case performance
    
    Returns:
    - best_alternative: str
        Index label of the best alternative
    """
    # Calculate metrics
    means = df.mean(axis=1)
    if len(df.columns) > 1:
        stds = df.std(axis=1)
    else:
        stds = pd.Series([0] * len(df.index), index=df.index)
    worst_cases = df.max(axis=1)
    
    # Composite score (lower is better)
    scores = means + std_penalty_weight * stds + worst_case_penalty_weight * worst_cases
    
    # Reporting
    report = pd.DataFrame({
        'Mean': means,
        'Std Dev': stds,
        'Worst Case': worst_cases,
        'Composite Score': scores
    }).sort_values('Composite Score')
    
    print("\n=== Alternative Evaluation Report ===\n")
    print(report)
    print("\n======================================\n")
    
    # Pick the best
    best_alternative = scores.idxmin()
    print(f"Selected Best Alternative: {best_alternative}")
    
    return best_alternative, report

def problem_parameters_definition(action: OpPlanActionType, initial_states: Optional[InitialStates] = None) -> ProblemParameters:
    
    if action == "startup":
        if debug_mode:
            # Simplify the combinations to have a reduced number of them
            operation_actions = {
                # Day 1 -----------------------  # Day 2 -----------------------
                "sfts": [("startup", 2), ("shutdown", 1), ("startup", 1), ("shutdown", 1)],
                "med": [("startup", 2), ("shutdown", 1), ("startup", 1), ("shutdown", 1)],
            }
        else:
            operation_actions = {
                # Day 1 -----------------------  # Day 2 -----------------------
                "sfts": [("startup", 3), ("shutdown", 2), ("startup", 1), ("shutdown", 1)],
                "med": [("startup", 3), ("shutdown", 2), ("startup", 1), ("shutdown", 1)],
            }
        irradiance_thresholds = IrradianceThresholds(min=300, max=600)
    
    elif action == "shutdown":
        # Shutdown operation updates
        if debug_mode:
            operation_actions= {
                # Day 1 ---------------  # Day 2 -----------------------
                "sfts": [("shutdown", 2), ("startup", 1), ("shutdown", 1)],
                "med":  [("shutdown", 2), ("startup", 1), ("shutdown", 1)],
            }
        else:
            # Shutdown operation updates
            operation_actions= {
                # Day 1 ---------------  # Day 2 -----------------------
                "sfts": [("shutdown", 3), ("startup", 2), ("shutdown", 2)],
                "med":  [("shutdown", 3), ("startup", 2), ("shutdown", 2)],
            }
        irradiance_thresholds = IrradianceThresholds(min=50, max=400)
        
    else:
        raise ValueError(f"Unknown action {action}. Options are: {get_args(OpPlanActionType)}")

    return ProblemParameters(
        optim_window_time=36 * 3600,  # 1d12h
        sample_time_opt=3600,  # 1h, In NLP-operation plan just used to resample environment variables
        operation_actions=operation_actions,
        initial_states=initial_states,
        real_dec_vars_update_period=RealDecisionVariablesUpdatePeriod(),
        initial_dec_vars_values=InitialDecVarsValues(), # Defaults valid for startup, not shutdown
        on_limits_violation_policy="penalize",
        irradiance_thresholds=irradiance_thresholds
    )

def build_archipielago(problems: list[BaseNlpProblem], algo_params: AlgoParams, x0: list[np.ndarray] = None, fitness0: list[float] = None) -> pg.archipelago:
    
    if x0 is not None:
        assert len(problems) == len(x0), f"Number of initial populations ({len(x0)}) should match number of problems ({len(problems)})"
        assert fitness0 is not None, "Initial fitness should be provided if initial populations are provided"
    
    archi = pg.archipelago()
    for problem_idx, problem in enumerate(problems):

        # Initialize problem instance
        prob = pg.problem(problem)
        
        # Initialize population
        pop = pg.population(prob, size=algo_params.pop_size, seed=0)
        if x0 is not None and x0[problem_idx] is not None:
            pop.set_xf(0, x0[problem_idx], [fitness0[problem_idx]])
        
        algo = pg.algorithm(getattr(pg, algo_params.algo_id)(**algo_params.params_dict))
        algo.set_verbosity( algo_params.log_verbosity )
        
        # 6. Build up archipielago
        archi.push_back(
            # Setting use_pool=True results in ever-growing memory footprint for the sub-processes
            # https://github.com/esa/pygmo2/discussions/168#discussioncomment-10269386
            pg.island(udi=pg.mp_island(use_pool=False), algo=algo, pop=pop, )
        )
        
    return archi

def evaluate_problems(problems: list[BaseNlpProblem], algo_params: AlgoParams, problems_eval_params: ProblemsEvaluationParameters, action: OpPlanActionType) -> OperationPlanResults:
    # This function should sequentially, build the archipielagos, 
    # evolve them, drop poorly performing problems and repeat until
    # the best performing problems are evolved completely
    
    def update_fitness_history(isl: pg.island, fit_his: pd.Series | None, initial: bool = False) -> pd.Series:
        
        if initial:
            fit = pd.Series(isl.get_population().champion_f[0], index=[0])
        else:
            fit = get_fitness_history(algo_params.algo_id, isl.get_algorithm() )
            fit_last = pd.Series(isl.get_population().champion_f[0], 
                                 index=[problems_eval_params.n_obj_fun_evals_per_update])
            if len(fit) == 0:
                fit = fit_last
            elif fit.index[-1] < problems_eval_params.n_obj_fun_evals_per_update:
                # Append the last fitness
                fit = pd.concat([fit, fit_last])
        
        if fit_his is None or len(fit_his) == 0:
            fit_his = fit
        else:
            fit.index = fit.index + fit_his.index[-1] +1
            fit_his = pd.concat([fit_his, fit])
            
        return fit_his
    
    date_str = list(asdict(problems[0].env_vars).values())[0].index[0].strftime("%Y%m%d")
    start_time = time.time()
    
    algo_params = AlgoParams(
        algo_id=algo_params.algo_id,
        max_n_obj_fun_evals=problems_eval_params.n_obj_fun_evals_per_update,
        pop_size=algo_params.pop_size,
        max_n_logs=algo_params.max_n_logs,
    )
    
    x = [None] * len(problems)
    fitness = [None] * len(problems)
    fitness_history = [None] * len(problems)
    droped_problem_idxs = []
    kept_problem_idxs = np.arange(len(problems))
    
    progress_bar = tqdm(range(problems_eval_params.n_updates), desc="Candidate problems evaluation", leave=True, position=2)
    
    for update_idx in progress_bar:
        
        # Evaluate problems for the update
        yet_to_eval_idxs = kept_problem_idxs
        batch_idx=1
        # log_header_str = f"{date_str} | Op.Plan - {action} | Evaluation step {update_idx+1}/{problems_eval_params.n_updates} | Active problems {len(kept_problem_idxs)}/{len(problems)}"
        
        while len(yet_to_eval_idxs) > 0:
            
            batch_size = min(problems_eval_params.max_n_parallel_problems, len(yet_to_eval_idxs))
            
            progress_bar.set_postfix(
                {"Active problems": f"{len(kept_problem_idxs)}/{len(problems)}"},
                {"Batch": f"{batch_idx}/{len(yet_to_eval_idxs)//batch_size+1}"},
            )
            
            archi = build_archipielago(
                problems=[problems[idx] for idx in yet_to_eval_idxs][:batch_size], 
                algo_params=algo_params,
                x0=[x[idx] for idx in yet_to_eval_idxs[:batch_size]],
                fitness0=[fitness[idx] for idx in yet_to_eval_idxs[:batch_size]],
            )
            # logger.info(f"{log_header_str} | Initialized archipelago of problems of size {batch_size}")
            
            # Add initial fitness to fitness history
            for idx, isl in enumerate(archi):
                problem_idx = yet_to_eval_idxs[idx]
                fitness_history[problem_idx] = update_fitness_history(isl, fitness_history[problem_idx], initial=True)
            
            # start_time2 = time.time()
            archi.evolve()
            # logger.info(archi)
            archi.wait_check()
            # while archi.status == pg.evolve_status.busy:
            #     time.sleep(5)
            #     print(f"Elapsed time: {time.time() - start_time:.0f}")
                # print(f"Current evolution results | Best fitness: {pop_current.champion_f[0]}, \nbest decision vector: {pop_current.champion_x}")
            
            # Update output objects
            for idx, isl in enumerate(archi):
                problem_idx = yet_to_eval_idxs[idx]
                x[problem_idx] = isl.get_population().champion_x
                fitness[problem_idx] = isl.get_population().champion_f[0]
                fitness_history[problem_idx] = update_fitness_history(isl, fitness_history[problem_idx])
            
            yet_to_eval_idxs = yet_to_eval_idxs[batch_size:]
            # logger.info(f"{log_header_str} | Completed evolution of batch {batch_idx}/{len(yet_to_eval_idxs)//batch_size+1}!. Took {int(time.time() - start_time2):.0f} seconds") 
            batch_idx+=1
        
        # Retain only the best performing problems
        fitness_current_update = np.array(copy.deepcopy(fitness))
        fitness_current_update[droped_problem_idxs] = np.nan
        kept_problem_idxs, drop_idxs = problems_eval_params.update_problems(fitness_current_update)
        droped_problem_idxs += drop_idxs
    
    evaluation_time = int(time.time() - start_time)
    longest_problem_x_idx = np.argmax([len(x_) for x_ in x])
    len_longest_x = len(x[longest_problem_x_idx])
    # dec_vec = [ for x_, problem in zip(x, problems)] # Including integer part
    # x should be padded with nans to match the length of the longest problem
    # fitness_history should be padded with nans to match lengths
    op_plan_results = OperationPlanResults(
        date_str=date_str, # Date in YYYYMMDD format
        action=action,
        x = pd.DataFrame(
            np.array([np.pad(item, (0, len_longest_x - len(item)), constant_values=np.nan) for item in x]), 
            columns = [
                f"{var_id}_step_{step_idx:03d}"
                for var_id, num_steps in asdict(problems[longest_problem_x_idx].dec_var_updates).items() if var_id not in problems[0].dec_var_int_ids
                for step_idx in range(num_steps)
            ]
        ),
        # int_dec_vars = [problem.int_dec_vars.to_dataframe() for problem in problems],
        fitness = pd.Series(fitness),
        fitness_history = pd.concat(fitness_history, axis=1).sort_index(),
        # environment_df = problems[0].env_vars.to_dataframe(),
        evaluation_time=evaluation_time,
        algo_params=algo_params,
        problems_eval_params=problems_eval_params,
    )
    
    # logger.info(f"Completed evolution process! Took {evaluation_time/60:.0f} minutes") 
    
    return op_plan_results

def evaluate_operation_plan_layer(problem_data: ProblemData, action: OpPlanActionType, uncertainty_factor: float = 0., ) -> tuple[list[OperationPlanResults], BaseNlpProblem, int]:
    
    if uncertainty_factor > 0:
        unc_factors = [uncertainty_factor, 0, -uncertainty_factor]
    else:
        unc_factors = [0]
        
    op_plan_results_list: list[OperationPlanResults] = []
    
    if debug_mode:
        algo_params = AlgoParams(max_n_obj_fun_evals=10,)
        problems_eval_params = ProblemsEvaluationParameters(
            drop_fraction=0.5,
            max_n_obj_fun_evals=algo_params.max_n_obj_fun_evals,
            n_obj_fun_evals_per_update=5
        )
    else:
        # Set default values for the algorithm and evaluation parameters
        algo_params = AlgoParams()
        problems_eval_params = ProblemsEvaluationParameters(n_updates=3, drop_fraction=0.5)
    
    progress_bar = tqdm(
        unc_factors, 
        desc=f"Op.Plan - {action} | Scenarios evaluation",
        leave=True, 
        position=1    # Position 1, main loop is at position 0
    )
    
    for scenario_idx, unc_factor in enumerate(progress_bar):
        progress_bar.set_postfix({"Uncertainty factor": f"{unc_factor:.2f}"})
        
        # Modify environment
        problem_data_copy = copy.deepcopy(problem_data)
        problem_data_copy.df["I"] = problem_data_copy.df["I"] * (1 + np.random.rand(len(problem_data.df)) * unc_factor)
        
        problems = initialize_problem_instances_nNLP(
            problem_data=problem_data_copy,        
            store_x=False,
            store_fitness=False,
            log=debug_mode
        )
        op_plan_results = evaluate_problems(
            problems,
            algo_params=algo_params,
            problems_eval_params=problems_eval_params,
            action=action,
        )
        op_plan_results.evaluate_best_problem(problems=problems, model=problem_data_copy.model)
        op_plan_results.scenario_idx = scenario_idx
        op_plan_results.problem_params = problem_data_copy.problem_params
        op_plan_results_list.append(op_plan_results)
        
    fitness_df = pd.concat([op_plan_results.fitness for op_plan_results in op_plan_results_list], axis=1)
    # Choose best alternative
    best_alternative_idx, _ = select_best_alternative(fitness_df)
    
    return op_plan_results_list, problems[best_alternative_idx], best_alternative_idx

def evaluate_operation_optimization_layer() -> None:
    raise NotImplementedError("Operation optimization layer evaluation not implemented yet")

def initialize_simulation(env_date_span_str: str, start_date: datetime.datetime, data_path: Path) -> tuple[ProblemData, EnvironmentVariables]:
    problem_params = problem_parameters_definition(action="startup", initial_states=get_initial_states())
    selected_date_span = (start_date, None)#start_date + datetime.timedelta(days=problem_params.optim_window_days)]

    problem_data = problem_initialization(
        problem_params=problem_params, 
        date_str=env_date_span_str, 
        data_path=data_path,
        selected_date_span=selected_date_span,
    )

    # Simulate idle system from environment start until some potential operation start
    env_vars = EnvironmentVariables.initialize_from_dataframe(
        problem_data.df, 
        # Not really needed just to estimate temperature decay 
        cost_w=problem_params.env_params.cost_w,
        cost_e=problem_params.env_params.cost_e
    )
    
    return problem_data, env_vars

def update_problem_data(pdata: ProblemData, start_dt: datetime.datetime, action: OpPlanActionType, sim_df: pd.DataFrame) -> ProblemData:
    
    
    pdata.df = pdata.df.loc[start_dt:]
    pdata.problem_params = problem_parameters_definition(action=action, initial_states=get_initial_states(sim_df))
    
    pp = pdata.problem_params
    pp.episode_duration = len(pdata.df) * pp.sample_time_mod
    pdata.problem_samples = times_to_samples(pp)
    
    if action == "shutdown":
        # Update initial decision variables values for operation plan - shutdown
        for fld in fields(InitialDecVarsValues):
            init_dec_var_vals_dict = {}
            if fld.name in OptimToFsmsVarIdsMapping()._asdict().keys():
                val = [sim_df[var_id].iloc[-1] > 0 for var_id in getattr( OptimToFsmsVarIdsMapping(), "sfts_mode" )]
            else:
                val = sim_df[fld.name].iloc[-1]
            init_dec_var_vals_dict[fld.name] = val
            
        pp.initial_dec_vars_values = InitialDecVarsValues(**init_dec_var_vals_dict)
    
    return pdata

def get_op_plan_evaluation_datetime(action: OpPlanActionType, computation_time: datetime.timedelta, I: pd.Series) -> datetime.datetime:
    
    pp = problem_parameters_definition(action=action)
    op_action_tuple = list(pp.operation_actions.values())[0][0]
    shutdown_dts = generate_update_datetimes(
        I, n=op_action_tuple[1], action_type=action, 
        irradiance_thresholds=pp.irradiance_thresholds
    )
    
    return pd.Series(shutdown_dts).min() - computation_time
    

# def intialize_op_plan_startup():
    
#     op_action_tuple = list(problem_params.operation_actions.values())[0][0]
#     startup_dts = generate_update_datetimes(env_vars.I, n=op_action_tuple[1], action_type=op_action_tuple[0], 
#                                             irradiance_thresholds=problem_params.irradiance_thresholds)
#     dt_span = [start_date, pd.Series(startup_dts).mean()]
#     # Update dt_span so that it matches available environment
#     dt_span[0] = max(env_vars.I.index[0], dt_span[0])
    
#     model = SolarMED(**problem_data.model.dump_instance()) # Avoid modifying the original model
#     Tts_h0, Tts_c0, sim_df = evaluate_idle_thermal_storage(model=model, dt_span=dt_span, env_vars=env_vars)
#     initial_states = get_initial_states(Tts_h=Tts_h0, Tts_c=Tts_c0)

#     return initial_states

# -----------------------------------------------------------------------------------------------------
# def main(date_span: tuple[str, str], data_path: Path, env_date_span: tuple[str, str], output_path: Path, uncertainty_factor: bool, operation_optimization_layer: bool) -> None:
logger.info(f"Evaluating nNLP-operation plan optimization for date span {date_span[0]}-{date_span[-1]}")

# Setup environment
env_date_span_str: str = f"{env_date_span[0]}_{env_date_span[1]}"
start_date = datetime.datetime.strptime(date_span[0], "%Y%m%d").replace(hour=0).astimezone(tz=datetime.timezone.utc)
end_date = datetime.datetime.strptime(date_span[1], "%Y%m%d").replace(hour=23).astimezone(tz=datetime.timezone.utc)
if date_span[0] == date_span[1]:
    all_dates = [start_date]
else:
    all_dates = list(pd.date_range(start=start_date, end=end_date, freq='D', tz='UTC'))
problem_data, env_vars = initialize_simulation(env_date_span_str=env_date_span_str, start_date=start_date, data_path=data_path)
model = problem_data.model
operation_end = env_vars.I.index[0]
sim_df = model.to_dataframe(index=operation_end)

# Setup progress bar
progress_bar = tqdm(all_dates, desc="SolarMED | Evaluating episode", unit="day", leave=True, position=0)
status_update_thread = threading.Thread(target=update_bar_every, args=[progress_bar, 20], daemon=True)
status_update_thread.start()
    
for date in progress_bar:
    progress_bar.set_postfix({"Current date": date.strftime("%Y%m%d")})
    
    # Update environment
    model.reset_fsms_cooldowns()
    env_vars = env_vars.dump_in_span(span=(operation_end, None), return_format="series")
    
    # Simulate idle system from last shutdown (if not first day) until start of operation plan evaluation
    startup_eval_datetime = get_op_plan_evaluation_datetime(
        action="startup",
        computation_time = datetime.timedelta(hours=3),
        I=env_vars.I.loc[env_vars.I.index > date]
    )
    _, _, sim_df = evaluate_idle_thermal_storage(
        model=model, 
        dt_span=(env_vars.I.index[0], startup_eval_datetime), 
        env_vars=env_vars, 
        df=sim_df,
        debug_mode=debug_mode,
    )
    if debug_mode:
        print(f"Current time: {sim_df.index[-1]} | After simulating from last day {operation_end=} up to {startup_eval_datetime=}")
    
    
    # Evaluate operation plan - startup
    problem_data = update_problem_data(problem_data, start_dt=startup_eval_datetime, action="startup", sim_df=sim_df)
    op_plan_results_list, problem, best_problem_idx = evaluate_operation_plan_layer(
        problem_data,
        uncertainty_factor=uncertainty_factor, 
        action="startup",
    )
    batch_export(output_path, op_plan_results_list)
    
    # Simulate up to operation optimization evaluation
    op_optim_eval_datetime = problem.operation_span[0] - datetime.timedelta(minutes=15)
    _, _, sim_df = evaluate_idle_thermal_storage(
        model=model, 
        dt_span=(startup_eval_datetime, op_optim_eval_datetime), 
        env_vars=env_vars, 
        df=sim_df,
        debug_mode=debug_mode,
    )
    
    if debug_mode:
        print(f"Current time: {sim_df.index[-1]} | After simulating from {startup_eval_datetime=} up to {op_optim_eval_datetime=}")
    
    # Get operation plan - shutdown evaluation datetime
    shutdown_eval_datetime = get_op_plan_evaluation_datetime(
        action="shutdown",
        computation_time = datetime.timedelta(minutes=30),
        I=env_vars.I.loc[:date.replace(hour=23, minute=59, second=59)] # Only current day, otherwise the last day in the environment will be chosen
    )
    
    # Evaluate operation optimization
    if not operation_optimization_layer:
        # From operation start to shutdown evaluation
        dt_span = [op_optim_eval_datetime, shutdown_eval_datetime]
        dec_vars = problem.decision_vector_to_decision_variables(
            x=op_plan_results_list[0].x.iloc[best_problem_idx].values # uncertainty scenario to be used: 0
        )
        env_vars_ = env_vars.dump_in_span(span=dt_span, return_format="series")
        sim_df = evaluate_model(
            model=model,
            mode="evaluation",
            dec_vars=dec_vars.dump_in_span(span=dt_span, return_format="series"),
            env_vars=env_vars_,
            # problem.env_vars are resampled to model sample time, maybe env_vars too?
            n_evals_mod=len(env_vars_.I),
            df_mod=sim_df,
            debug_mode=debug_mode,
        )
        if debug_mode:
            print(f"Current time: {sim_df.index[-1]} | After simulating from operation start ({op_optim_eval_datetime}) to shutdown evaluation ({shutdown_eval_datetime})")
    # else:
    # Do it in a separate thread since operation optimization and, eventual evaluation
    # of operation plan - shutdown should be evaluated in parallel
    # 
    # int_dec_vars = problem.int_dec_vars
    # evaluate_operation_optimization_layer(
    #     x0 = [op_plan_results.x[] for op_plan_results in op_plan_results_list]
    # )
    
    # Evaluate operation plan - shutdown
    problem_data = update_problem_data(problem_data, start_dt=shutdown_eval_datetime, action="shutdown", sim_df=sim_df)
    if debug_mode:
        print(f"Current time in problem_data.df: {problem_data.df.index[0]} | After updating problem data in order to evaluate op.plan-shutdown")
    
    op_plan_results_list, problem, best_problem_idx = evaluate_operation_plan_layer(
        problem_data=problem_data,
        uncertainty_factor=0, 
        action="shutdown",
    )
    batch_export(output_path, op_plan_results_list)
    
    # Simulate from shutdown evaluation up to shutdown completion
    dt_span = [shutdown_eval_datetime, problem.operation_span[-1]]
    sim_df = evaluate_model(
        model=model,
        mode="evaluation",
        dec_vars=dec_vars.dump_in_span(span=dt_span, return_format="series"),
        env_vars=env_vars.dump_in_span(span=dt_span, return_format="series"),
        # problem.env_vars are resampled to model sample time, maybe env_vars too?
        n_evals_mod=len(problem.env_vars.dump_in_span(span=dt_span, return_format="series").I),
        df_mod=sim_df,
        evaluate_shutdown=True,
        debug_mode=debug_mode,
    )
    # Update operation end to actual value after simulation
    operation_end = sim_df.index[-1]
    
    # Export simulation results
    # ...


2025-05-04 09:33:34.474 | INFO     | __main__:<module>:400 - Evaluating nNLP-operation plan optimization for date span 20180921-20180922


2025-05-04 09:33:37.734 | WARNING  | solarmed_optimization.utils.initialization:problem_initialization:107 - External environment data (20220524 - 20240524) (seawater temperature and salinity) is not available for the data period (20180921 - 20180928). Using data for same month and days but from closest year 2022.
2025-05-04 09:33:37.740 | INFO     | solarmed_optimization.utils.initialization:problem_initialization:119 - Selected date span: 2018-09-21 00:00:00+00:00 - None from 2018-09-21 05:00:00+00:00 - 2018-09-28 00:00:00+00:00


SolarMED | Evaluating episode:   0%|          | 0/2 [00:00<?, ?day/s]

Op.Plan - startup | Scenarios evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Candidate problems evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

...

2025-05-04 12:41:11.290 | INFO     | solarmed_optimization.problems.nlp:export:221 - Exported results to /workspaces/SolarMED/optimization/results/20180921_20180922/results_nNLP_op_plan_eval_at_20250504T0930_dev.h5 / /20180921/startup/scenario_00



=== Alternative Evaluation Report ===

          Mean  Std Dev  Worst Case  Composite Score
33 -128.135138        0 -128.135138      -256.270277
12 -124.956341        0 -124.956341      -249.912682
5  -123.968672        0 -123.968672      -247.937343
26 -123.573254        0 -123.573254      -247.146509
38 -122.670714        0 -122.670714      -245.341427
19 -122.058661        0 -122.058661      -244.117321
31 -120.304923        0 -120.304923      -240.609846
17 -120.001424        0 -120.001424      -240.002848
24 -119.816995        0 -119.816995      -239.633990
3  -118.026862        0 -118.026862      -236.053723
10 -117.182651        0 -117.182651      -234.365301
4  -116.102643        0 -116.102643      -232.205286
18 -115.282953        0 -115.282953      -230.565905
39 -115.238833        0 -115.238833      -230.477665
1  -113.649797        0 -113.649797      -227.299593
8  -113.050310        0 -113.050310      -226.100620
40 -112.899530        0 -112.899530      -225.799059
15 -11

Op.Plan - shutdown | Scenarios evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

duplicated in 0: DatetimeIndex(['2018-09-22 16:50:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
duplicated in 1: DatetimeIndex(['2018-09-22 16:50:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
duplicated in 2: DatetimeIndex(['2018-09-22 16:50:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
duplicated in 3: DatetimeIndex(['2018-09-22 16:50:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
duplicated in 4: DatetimeIndex(['2018-09-22 16:50:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
duplicated in 5: DatetimeIndex(['2018-09-22 16:50:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
duplicated in 6: DatetimeIndex(['2018-09-22 16:50:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
duplicated in 7: DatetimeIndex(['2018-09-22 16:50:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
duplicated in 8: DatetimeIndex(['2018-09-22 16:50:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
duplicated in 9: DatetimeIndex(['2018-09-22 16:50:00+00:00'], dtype='datetime64[ns

ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
# Prototype op.plan results visualization



In [ ]:
env_vars.I


time
2018-09-21 16:53:20+00:00    448.599408
2018-09-21 17:00:00+00:00    424.634685
2018-09-21 17:06:40+00:00    395.795468
2018-09-21 17:13:20+00:00    368.878125
2018-09-21 17:20:00+00:00    341.190787
                                ...    
2018-09-27 23:20:00+00:00      0.000000
2018-09-27 23:26:40+00:00      0.000000
2018-09-27 23:33:20+00:00      0.000000
2018-09-27 23:40:00+00:00      0.000000
2018-09-27 23:46:40+00:00      0.000000
Freq: 400s, Name: I, Length: 1359, dtype: float64

In [ ]:
# Test importing-exporting operation plan results multiple times
from solarmed_optimization.problems.nlp import batch_export

test_path = Path("/workspaces/SolarMED/optimization/results/20180921_20180921/results_nNLP_op_plan_eval_at_20250429T1811_dev.gz")

op_plan_results_list2 = [
	OperationPlanResults.initialize(
		input_path=test_path, # output_path.with_suffix(".gz"), 
		date_str="20180921",
		action="startup",
		scenario_idx=scenario_idx,
	)
	for scenario_idx in range(3)
]


test_path = test_path.with_stem("test")
batch_export(test_path, op_plan_results_list2)

[
	OperationPlanResults.initialize(
		input_path=test_path, # output_path.with_suffix(".gz"), 
		date_str="20180921",
		action="startup",
		scenario_idx=scenario_idx,
	)
	for scenario_idx in range(3)
]


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [ ]:
# TODO: Create a OpPlanVisualizer that given a OpPlanResults object has methods
# to plot:
# - Decision variables evolution
# - Timeseries results
# - Problems fitness evolution
# - 
